# None of this was used in the final model.  It's here just as part of the record

# Importing saved bigrams

In [ ]:
from joblib import load, dump
dfex_en=load('bigrams_final.joblib')

In [ ]:
import pandas as pd

# Vectorize

## Converting to lower case

In [ ]:
dfex_en.head()

,datetime,lang,rts,likes,quotes,replies,hr,date,bigs
0,2020-03-01T20:43:33.000Z,en,0.0,1.0,0.0,1,20.0,2020-03-01,"[(@ronnie_haydon, Almost), (Almost, certainly)..."
1,2020-03-01T17:03:47.000Z,en,0.0,0.0,0.0,0,17.0,2020-03-01,"[([, Ivermectin), (Ivermectin, Paste), (Paste,..."
2,2020-03-01T12:15:08.000Z,en,0.0,8.0,0.0,0,12.0,2020-03-01,"[(🏁, The), (The, BOHEMIA), (BOHEMIA, team), (t..."
3,2020-03-02T15:25:11.000Z,en,1.0,4.0,0.0,0,15.0,2020-03-02,"[(@ginagh, works), (works, as), (as, #ISGlobal..."
4,2020-03-03T21:00:33.000Z,en,0.0,0.0,0.0,1,21.0,2020-03-03,"[(@LiezJKC, Categorically), (Categorically, un..."


In [ ]:
lower=[]

for j in dfex_en.index:
  if j%500000==0:
    pd.DataFrame(lower).to_csv('lower'+str(j)+'.csv')
  lower.append([j, [(str(x).lower(), str(y).lower()) for (x, y) in dfex_en.bigs.iloc[j]]])

In [ ]:
pd.DataFrame(lower).to_csv('lower_final.csv')

In [ ]:
lower=pd.DataFrame(lower)
lower=pd.read_csv('lower_final.csv', index_col='Unnamed: 0')
dump(dfex_en.drop(['bigs'], axis=1),'non-text_df.joblib')

In [ ]:
dfex_en.drop(['bigs'], axis=1,inplace=True)
dfex_en['bigs']=lower[1]
dump(dfex_en, 'bigrams_lower.joblib')

## Vectorizing

To feed tokens to tfidfvectorizer, can create dummy tokenizer and preprocessor
https://www.davidsbatista.net/blog/2018/02/28/TfidfVectorizer/

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def dummy_fun(doc):
  return doc

In [ ]:
vectorizer=TfidfVectorizer(
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern = None,
    max_features=5000
    )

In [ ]:
dfex_en

,datetime,lang,rts,likes,quotes,replies,hr,date,bigs
0,2020-03-01T20:43:33.000Z,en,0.0,1.0,0.0,1,20.0,2020-03-01,"[(@ronnie_haydon, Almost), (Almost, certainly)..."
1,2020-03-01T17:03:47.000Z,en,0.0,0.0,0.0,0,17.0,2020-03-01,"[([, Ivermectin), (Ivermectin, Paste), (Paste,..."
2,2020-03-01T12:15:08.000Z,en,0.0,8.0,0.0,0,12.0,2020-03-01,"[(🏁, The), (The, BOHEMIA), (BOHEMIA, team), (t..."
3,2020-03-02T15:25:11.000Z,en,1.0,4.0,0.0,0,15.0,2020-03-02,"[(@ginagh, works), (works, as), (as, #ISGlobal..."
4,2020-03-03T21:00:33.000Z,en,0.0,0.0,0.0,1,21.0,2020-03-03,"[(@LiezJKC, Categorically), (Categorically, un..."
...,...,...,...,...,...,...,...,...,...
2715892,2022-10-13T00:01:14.000Z,en,0.0,4.0,0.0,0,0.0,2022-10-13,"[(@KevinCTV, So), (So, why), (why, are), (are,..."
2715893,2022-10-13T00:01:06.000Z,en,0.0,0.0,0.0,0,0.0,2022-10-13,"[(@ElijahSchaffer, Im), (Im, alive), (alive, ...."
2715894,2022-10-13T00:00:40.000Z,en,0.0,1.0,0.0,0,0.0,2022-10-13,"[(@makeupartist524, Go), (Go, here), (here, an..."
2715895,2022-10-13T00:00:29.000Z,en,1.0,1.0,0.0,0,0.0,2022-10-13,"[(@MelissaMbarki, The), (The, policies), (poli..."


In [ ]:
vecs = vectorizer.fit_transform(dfex_en.bigs)

from joblib import dump
dump(vectorizer, 'vectorizer.joblib')

## Importing from previous

In [ ]:
vectorizer=load('vectorizer.joblib')

In [ ]:
vecs=vectorizer.transform(dfex_en.bigs)
dump(vecs, 'vecs.joblib')

In [ ]:
vecs_df = pd.DataFrame.sparse.from_spmatrix(vecs)

In [ ]:
vecs_df.columns = vectorizer.vocabulary_

In [ ]:
vecs_df

,i,to,see,if,they,happy,to,give,some,ivermectin,...,rogan,aaron,india,announces,free,200,@drsanjaygupta,against,analysis,-
,would,see,if,they,are,to,give,you,ivermectin,to,...,took,rodgers,announces,state,proving,members,@joerogan,omicron,know,https://t.co/3fpvmuvpkt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2715892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2715893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2715894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
vecs_df.columns

MultiIndex([(             'i',                   'would'),
            (            'to',                     'see'),
            (           'see',                      'if'),
            (            'if',                    'they'),
            (          'they',                     'are'),
            (         'happy',                      'to'),
            (            'to',                    'give'),
            (          'give',                     'you'),
            (          'some',              'ivermectin'),
            (    'ivermectin',                      'to'),
            ...
            (         'rogan',                    'took'),
            (         'aaron',                 'rodgers'),
            (         'india',               'announces'),
            (     'announces',                   'state'),
            (          'free',                 'proving'),
            (           '200',                 'members'),
            ('@drsanjaygupta',          

# Modeling

from https://www.analyticsvidhya.com/blog/2021/05/topic-modelling-in-natural-language-processing/

### Loading previous data

In [ ]:
vecs = pd.read_csv('vecs.csv', index_col = 'Unnamed: 0')

In [ ]:
from joblib import load
vectorizer = load('vectorizer.joblib')
tokes = pd.read_csv('tokes_no_punc_no_stop_lower.csv', index_col = 'Unnamed: 0')
vecs = vectorizer.transform(tokes.lower)

### Writing Vectorizer Vocab to a file

In [ ]:
with open('vocab1.txt', 'w') as f:
    for (x,y) in vecs_df.columns:
      f.write(str(x)+', '+str(y)+'\n')

In [ ]:
vecs_df

,i,to,see,if,they,happy,to,give,some,ivermectin,...,rogan,aaron,india,announces,free,200,@drsanjaygupta,against,analysis,-
,would,see,if,they,are,to,give,you,ivermectin,to,...,took,rodgers,announces,state,proving,members,@joerogan,omicron,know,https://t.co/3fpvmuvpkt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2715892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2715893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2715894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Trying single model and partial fitting, rather than Grid Search

In [ ]:
# Parameters tuning using Grid Search
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

# LDA model
lda = LatentDirichletAllocation(
    n_components = 10, 
    learning_method='online',
    n_jobs = -1)
lda.fit(vecs_df) #default is 1000000 samples


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


LatentDirichletAllocation(learning_method='online', n_jobs=-1)

In [ ]:
dump(lda, 'lda.joblib')

# Training different vectorizer to remove frequent bigrams w/o meanings

In [ ]:
vectorizer_lim=TfidfVectorizer(
    tokenizer=dummy_fun, #TweetTokenizer, did not opt to remove handles
    preprocessor=dummy_fun,
    token_pattern = None,
    max_features=5000,
    max_df=0.60 #v2 = 0.8, v3 = 0.5, v6 = 0.4, v7 = 0.6
    #min_df=0.0001 #v4 = 0.00001, v5 0.0001, v8 = taking this out again
    )

max_df of 0.5 might be too high.

In [ ]:
vecs = vectorizer_lim.fit_transform(dfex_en.bigs)

from joblib import dump
dump(vectorizer_lim, 'vectorizer_v8.joblib')

In [ ]:
with open('vec_v8_vocab.txt', 'w') as f:
    for (x,y) in vectorizer_lim.vocabulary_:
      f.write(str(x)+', '+str(y)+'\n')

In [ ]:
vectorizer_lim.vocabulary_

{('I', 'would'): 673,
 ('to', 'see'): 4356,
 ('see', 'if'): 3659,
 ('if', 'they'): 2494,
 ('they', 'are'): 4162,
 ('happy', 'to'): 2310,
 ('to', 'give'): 4305,
 ('give', 'you'): 2264,
 ('some', 'ivermectin'): 3724,
 ('ivermectin', 'to'): 2817,
 ('to', 'treat'): 4376,
 ('will', 'do'): 4749,
 ('do', 'this'): 1920,
 ('this', 'for'): 4229,
 ('for', 'free'): 2149,
 ('free', '.'): 2204,
 ('it', 'in'): 2724,
 ('for', 'her'): 2150,
 ('to', 'eat'): 4295,
 ('where', 'you'): 4700,
 ('the', 'same'): 4082,
 ('same', 'time'): 3625,
 ('has', 'been'): 2317,
 ('-', 'Ivermectin'): 272,
 ('up', 'to'): 4489,
 ('to', 'start'): 4361,
 ('of', 'a'): 3187,
 (')', '&'): 85,
 ('in', 'the'): 2567,
 ('through', 'the'): 4256,
 ('based', 'on'): 1573,
 ('on', '#ivermectin'): 3263,
 ('#ivermectin', '.'): 39,
 ('time', 'to'): 4262,
 ('works', 'as'): 4826,
 ('of', 'the'): 3237,
 ('and', 'as'): 1326,
 ('to', 'reduce'): 4353,
 ('is', 'only'): 2656,
 ('away', 'from'): 1566,
 ('are', 'still'): 1495,
 ('-', 'ivermectin'): 28

In [ ]:
vecs_df = pd.DataFrame.sparse.from_spmatrix(vecs)
vecs_df.columns=vectorizer_lim.vocabulary_

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# LDA model
lda = LatentDirichletAllocation(
    n_components = 12, 
    learning_method='online',
    max_iter=4,
    random_state=19,
    n_jobs = -1,
    verbose=1)
lda.fit(vecs_df) #default is 1000000 samples

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


iteration: 1 of max_iter: 4
iteration: 2 of max_iter: 4
iteration: 3 of max_iter: 4
iteration: 4 of max_iter: 4


LatentDirichletAllocation(learning_method='online', max_iter=4, n_components=12,
                          n_jobs=-1, random_state=19, verbose=1)

In [ ]:
dump(lda, 'lda_t12_vec_v8.joblib')

In [ ]:
components=pd.DataFrame(lda.components_.T, index=vectorizer_lim.vocabulary_)

In [ ]:
top_20=[]

for i in range(0,12):
  top_20.append(list(components[i].sort_values(ascending=False)[:30].index))

In [ ]:
pd.DataFrame(top_20).T

,0,1,2,3,4,5,6,7,8,9,10,11
0,"(#SackSAGE, #SackPM)","(over, the)","(the, market)","(look, like)","(people, should)","(but, ivermectin)","(case, ,)","(to, our)","(have, no)","(prescribing, Ivermectin)","(part, of)","(ivermectin, that)"
1,"(,, etc)","(,, at)","(the, Queen)","(would, have)","(not, take)","(is, probably)","(and, more)","(Treat, COVID)","(an, ivermectin)","(doctor, to)","(via, @YouTube)","(I, would)"
2,"(now, ,)","(in, vitro)","(ivermectin, I)","(drug, Ivermectin)","(tell, me)","(#SackSAGE, #SackPM)","(19, "")","(about, ?)","(in, ivermectin)","(t, make)","(..., the)","(for, covid)"
3,"(Treatment, of)","(know, of)","(effective, treatments)","(your, body)","(ivermectin, studies)","(cure, for)","(,, "")","(media, ,)","(with, Ivermectin)","(should, be)","(is, just)","(are, trying)"
4,"(as, long)","(Ivermectin, kills)","(Of, course)","(over, the)","(brave, and)","(de-fenestration, of)","(It, ’)","(vax, is)","(on, heavy)","(and, should)","(they, won)","(to, find)"
5,"(dies, from)","(Tractor, Supply)","(19, .)","(,, at)","(treating, covid)","(in, horse)","(the, jab)","(social, media)","(the, unvaccinated)","(,, do)","(it, does)","(But, you)"
6,"(,, then)","(is, over)","(the, next)","(medicines, .)","(2020, .)","(and, doctors)","(are, killing)","(), are)","(much, better)","(Effectiveness, of)","(it, was)","(t, we)"
7,"(prevent, COVID)","(get, ivermectin)","(taking, ivermectin)","(ivermectin, that)","(being, treated)","(part, of)","(efficacy, in)","(a, lot)","(with, them)","(I, have)","(I, do)","(ivermectin, in)"
8,"(said, ,)","(,, D)","(on, them)","(Ivermectin, not)","(and, not)","(against, the)","(Covid, ?)","(in, some)","(was, in)","(we, ’)","(that, "")","("", Deworming)"
9,"(to, talk)","(with, these)","(de-fenestration, of)","(good, ,)","(as, much)","(hasn't, been)","(just, a)","(Andrew, Hill)","(info, .)","(,, an)","(and, others)","(all, about)"


# Grid Searching number of topics

In [ ]:
# Parameters tuning using Grid Search
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

just_vecs = vecs_df
grid_params = {'n_components' : list(range(5,10)), 'n_jobs':[-1]}
# LDA model
lda = LatentDirichletAllocation()
lda_model = GridSearchCV(lda,param_grid=grid_params)
lda_model.fit(just_vecs)
# Estimators for LDA model
lda_model1 = lda_model.best_estimator_

dump(lda_model1, 'ldamodel1.joblib')

print("Best LDA model's params" , lda_model.best_params_)
print("Best log likelihood Score for the LDA model",lda_model.best_score_)
print("LDA model Perplexity on train data", lda_model1.perplexity(just_vecs))

# Trigram Modeling

In [ ]:
tokens=load('trigrams.joblib')

In [ ]:
tokens.head()

,flat
0,"[(@ronnie_haydon, Almost, certainly), (Almost,..."
1,"[([, Ivermectin, Paste), (Ivermectin, Paste, D..."
2,"[(🏁, The, BOHEMIA), (The, BOHEMIA, team), (BOH..."
3,"[(@ginagh, works, as), (works, as, #ISGlobal),..."
4,"[(@LiezJKC, Categorically, untrue), (Categoric..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer_lim=TfidfVectorizer(
    tokenizer=dummy_fun, #TweetTokenizer, did not opt to remove handles
    preprocessor=dummy_fun,
    token_pattern = None,
    max_features=6000, #v10 inc to 6000
    max_df=0.70 #v2 = 0.8, v3 = 0.5, v6 = 0.4, v7 = 0.6, v9 = 0.7
    #min_df=0.0001 #v4 = 0.00001, v5 0.0001, v8 = taking this out again
    )

## Vectorizer with trigrams

In [ ]:
vecs = vectorizer_lim.fit_transform(tokens.flat)

#as you can see this is the test of the 10th version, use whatever you named your vectorizer here
dump(vectorizer_lim, 'vectorizer_v10.joblib')

In [ ]:
with open('vec_v10_vocab.txt', 'w') as f:
    for (x,y,z) in vectorizer_lim.vocabulary_:
      f.write(str(x)+', '+str(y)+'\n')

In [ ]:
vectorizer_lim.vocabulary_

{('to', 'see', 'if'): 5289,
 ('if', 'they', 'are'): 2923,
 ('to', 'give', 'you'): 5206,
 ('ivermectin', 'to', 'treat'): 3489,
 ('the', 'same', 'time'): 4935,
 ('in', 'the', 'fight'): 3037,
 ('the', 'fight', 'against'): 4826,
 ('some', 'Ivermectin', '.'): 4449,
 ('to', 'be', 'an'): 5132,
 ('be', 'an', 'effective'): 1879,
 ('for', 'some', 'people'): 2535,
 ('a', 'number', 'of'): 1353,
 ('a', 'drug', 'that'): 1296,
 ('on', 'Ivermectin', 'and'): 3985,
 ('in', 'a', 'few'): 2979,
 ('a', 'few', 'weeks'): 1304,
 ('horse', 'paste', '.'): 2849,
 ('my', 'immune', 'system'): 3706,
 ('it', 'out', '.'): 3330,
 ('I', 've', 'been'): 602,
 ('couldn', 't', 'get'): 2102,
 ('just', 'take', 'ivermectin'): 3515,
 ('I', 'had', 'the'): 529,
 ('as', 'well', '.'): 1835,
 ('so', 'I', 'can'): 4433,
 ('to', 'try', 'to'): 5352,
 ('I', 'live', 'in'): 563,
 ('the', 'ones', 'that'): 4895,
 ('dose', 'of', 'ivermectin'): 2277,
 ('of', 'ivermectin', 'for'): 3891,
 ('very', 'effective', '.'): 5567,
 ('horse', 'dewormer', 

In [ ]:
vecs_df = pd.DataFrame.sparse.from_spmatrix(vecs)
vecs_df.columns=vectorizer_lim.vocabulary_

## Modeling with trigrams

### starting from previous

In [ ]:
def dummy_fun(tokes):
  return tokes

In [ ]:
vectorizer_lim = load('v10_vecs.joblib')
vecs=vectorizer_lim.transform(tokens.flat)

In [ ]:
vecs_df = pd.DataFrame.sparse.from_spmatrix(vecs)
vecs_df.columns=vectorizer_lim.vocabulary_

In [ ]:
vecs_df.head()

to   if   to ivermectin  the    in     the       some   to        be  ...  \
   see they give         to same   the   fight Ivermectin   be        an  ...   
    if  are  you      treat time fight against          .   an effective  ...   
0  0.0  0.0  0.0        0.0  0.0   0.0     0.0        0.0  0.0       0.0  ...   
1  0.0  0.0  0.0        0.0  0.0   0.0     0.0        0.0  0.0       0.0  ...   
2  0.0  0.0  0.0        0.0  0.0   0.0     0.0        0.0  0.0       0.0  ...   
3  0.0  0.0  0.0        0.0  0.0   0.0     0.0        0.0  0.0       0.0  ...   
4  0.0  0.0  0.0        0.0  0.0   0.0     0.0        0.0  0.0       0.0  ...   

    it Years     👉       Prize Winner    👉          UP  C19  
   reg   old Nobel   is Winner      👉   is    UP India free  
     .  SAFE Prize   an      👉     is    a India   C19    👉  
0  0.0   0.0   0.0  0.0    0.0    0.0  0.0   0.0   0.0  0.0  
1  0.0   0.0   0.0  0.0    0.0    0.0  0.0   0.0   0.0  0.0  
2  0.0   0.0   0.0  0.0    0.0    0.0  0.0   0.0   0.0  0.0  
3  0.0   0.0   0.0  0.0    0.0    0.0  0.0   0.0   0.0  0.0  
4  0.0   0.0   0.0  0.0    0.0    0.0  0.0   0.0   0.0  0.0  

[5 rows x 6000 columns]

In [ ]:
dump(vecs,'v10_vecs.joblib')

## training model using sparse matrix

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# LDA model
lda = LatentDirichletAllocation(
    n_components = 20, 
    learning_method='online',
    max_iter=4,
    random_state=19,
    n_jobs = 19,
    verbose=1)
lda.fit(vecs) #default is 1000000 samples

iteration: 1 of max_iter: 4
iteration: 2 of max_iter: 4
iteration: 3 of max_iter: 4
iteration: 4 of max_iter: 4


LatentDirichletAllocation(learning_method='online', max_iter=4, n_components=20,
                          n_jobs=19, random_state=19, verbose=1)

In [ ]:
from joblib import dump
dump(lda, 'lda_t20_vec_v10.joblib')

['lda_t20_vec_v10.joblib']

In [ ]:
components=pd.DataFrame(lda.components_.T, index=vectorizer_lim.vocabulary_)

In [ ]:
top_20=[]

for i in range(0,20):
  top_20.append(list(components[i].sort_values(ascending=False)[:30].index))

In [ ]:
pd.DataFrame(top_20).T.to_csv('topics_20.csv')

# Trying Gensim LDA models

## Installing gensim

In [ ]:
%pip install gensim

In [ ]:
import gensim
from gensim.models import LdaMulticore, CoherenceModel

## Importing

In [ ]:
import pandas as pd

In [ ]:
from joblib import load
tokens=load("trigrams.joblib")

In [ ]:
tokens.head()

,flat
0,"[(@ronnie_haydon, Almost, certainly), (Almost,..."
1,"[([, Ivermectin, Paste), (Ivermectin, Paste, D..."
2,"[(🏁, The, BOHEMIA), (The, BOHEMIA, team), (BOH..."
3,"[(@ginagh, works, as), (works, as, #ISGlobal),..."
4,"[(@LiezJKC, Categorically, untrue), (Categoric..."


In [ ]:
temp=tokens.head(3)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer_lim=TfidfVectorizer(
    tokenizer=dummy_fun, #TweetTokenizer, did not opt to remove handles
    preprocessor=dummy_fun,
    token_pattern = None,
    max_features=6000, #v10 inc to 6000
    max_df=0.70 #v2 = 0.8, v3 = 0.5, v6 = 0.4, v7 = 0.6, v9 = 0.7
    #min_df=0.0001 #v4 = 0.00001, v5 0.0001, v8 = taking this out again
    )

In [ ]:
vecs=vectorizer_lim.fit_transform(tokens.flat)

## Reference Gensim Code

In [ ]:
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("model")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [ ]:
other_texts = [
    ['computer', 'time', 'graph'],
    ['survey', 'response', 'eps'],
    ['human', 'system', 'computer']
]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[0]
vector = lda[unseen_doc]  # get topic probability distribution for a document

# Update the model by incrementally training on the new corpus.
lda.update(other_corpus)  # update the LDA model with additional documents

## modelling

In [ ]:
id_dict = dict(zip(vectorizer_lim.vocabulary_.values(), vectorizer_lim.vocabulary_.keys()))

In [ ]:
lda = LdaMulticore(vecs, id2word=id_dict, num_topics=8)
lda.save('gs_lda.model')

KeyboardInterrupt: 

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [ ]:
vis_data = gensimvis.prepare(lda, vecs, vectorizer_lim.vocabulary_)
pyLDAvis.display(vis_data)